# MGP and MI exchange volumes
The dataset *GME_MGP_MI_QUANTITA* refers to the total volume traded (both sell and buy) for each hours in the MGP and MI markets.

In [1]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])

volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,FLOWDATE,FLOWHOUR,MARKET,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,...,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME,FLOWQUARTER,TIMERESOLUTION
80766,20190101,1,MI1,1207.748,858.713,83.695,68.004,827.752,8.921,44.549,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,2018-12-31 15:10:52.263,NaN,None
80778,20190101,1,MGP,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,...,2438.0,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097,NaN,None
80824,20190101,1,MI2,244.328,403.747,8.931,49.858,72.777,4.295,32.767,...,0.0,0.0,0.0,NaN,NaN,412.328,412.328,2018-12-31 16:41:34.837,NaN,None
80767,20190101,2,MI1,1262.958,913.156,77.863,104.702,896.259,5.944,27.843,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,2018-12-31 15:10:52.267,NaN,None
80779,20190101,2,MGP,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,...,2342.0,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,20241213,23,MI1,1803.686,1535.276,122.246,303.615,832.809,4.146,84.446,...,45.0,0.0,0.0,0.0,0.0,NaN,NaN,2024-12-12 15:50:52.803,23.0,PT60
1270,20241213,24,MI3,702.918,693.828,15.871,42.916,448.869,154.366,18.840,...,50.0,16.7,0.0,28.9,0.0,NaN,NaN,2024-12-13 10:31:09.853,24.0,PT60
1274,20241213,24,MI2,639.233,634.873,20.057,73.157,273.051,21.594,48.601,...,10.0,0.0,0.0,0.0,0.0,NaN,NaN,2024-12-12 22:30:42.533,24.0,PT60
1436,20241213,24,MI1,1409.818,1348.838,153.513,196.622,666.004,5.822,90.237,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2024-12-12 15:50:52.803,24.0,PT60


In [2]:
market_labels = volumes['MARKET'].unique()

print(market_labels)

['MI1' 'MGP' 'MI2' 'MI3' 'MI4' 'MI5' 'MI6' 'MI7']


In [3]:
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

In [4]:
mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME,FLOWQUARTER,TIMERESOLUTION
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097,NaN,None
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097,NaN,None
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097,NaN,None
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100,NaN,None
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-13 19:00:00,43210.855,44069.484,3675.566,7793.277,23323.551,1174.003,2364.969,2613.840,0.0,None,...,477.000,0.0,0.0,0.0,0.0,44010.515,43510.555,2024-12-12 13:10:49.317,20.0,PT60
2024-12-13 20:00:00,40943.994,40943.994,3403.500,7425.759,21669.275,1153.128,2320.965,2515.856,0.0,None,...,784.000,0.0,0.0,0.0,625.0,40172.271,40943.994,2024-12-12 13:10:49.317,21.0,PT60
2024-12-13 21:00:00,37181.938,37181.938,3108.442,6828.312,20012.009,1075.712,2164.911,2326.693,0.0,None,...,1623.000,0.0,0.0,0.0,533.0,36149.383,37181.938,2024-12-12 13:10:49.317,22.0,PT60


In [5]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mgp_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-12-12 18:00:00,24497.015,28797.957,2024-12-11 13:11:06.217
2024-12-12 19:00:00,23920.596,27631.057,2024-12-11 13:11:06.217
2024-12-12 20:00:00,22439.003,23868.814,2024-12-11 13:11:06.217
2024-12-12 21:00:00,20897.472,21135.586,2024-12-11 13:11:06.217
2024-12-12 22:00:00,18914.351,18833.972,2024-12-11 13:11:06.217
2024-12-12 23:00:00,17462.457,14834.036,2024-12-11 13:11:06.217
2024-12-13 00:00:00,16310.026,13419.696,2024-12-12 13:10:49.307
2024-12-13 01:00:00,15708.768,12492.817,2024-12-12 13:10:49.310
2024-12-13 02:00:00,15609.696,11256.313,2024-12-12 13:10:49.313


In [6]:
mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
mi1_volumes = mi1_volumes.drop(columns=['MARKET'])

In [7]:
mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi1_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-12-12 18:00:00,420.763,627.286,2024-12-11 15:31:07.193
2024-12-12 19:00:00,365.548,858.767,2024-12-11 15:31:07.193
2024-12-12 20:00:00,425.526,373.250,2024-12-11 15:31:07.193
2024-12-12 21:00:00,387.447,304.341,2024-12-11 15:31:07.193
2024-12-12 22:00:00,745.367,697.131,2024-12-11 15:31:07.193
2024-12-12 23:00:00,666.439,697.221,2024-12-11 15:31:07.193
2024-12-13 00:00:00,505.470,636.731,2024-12-12 15:50:52.793
2024-12-13 01:00:00,761.541,698.914,2024-12-12 15:50:52.793
2024-12-13 02:00:00,791.581,704.058,2024-12-12 15:50:52.797


In [8]:
mi2_volumes = volumes[volumes['MARKET'] == 'MI2']
mi2_volumes = mi2_volumes.drop(columns=['MARKET'])

In [9]:
mi2_volumes_nord = mi2_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi2_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-12-12 18:00:00,258.912,325.940,2024-12-11 22:30:44.520
2024-12-12 19:00:00,370.184,301.373,2024-12-11 22:30:44.520
2024-12-12 20:00:00,115.559,141.658,2024-12-11 22:30:44.520
2024-12-12 21:00:00,125.499,113.623,2024-12-11 22:30:44.520
2024-12-12 22:00:00,76.051,60.214,2024-12-11 22:30:44.520
2024-12-12 23:00:00,174.331,103.089,2024-12-11 22:30:44.520
2024-12-13 00:00:00,245.017,234.468,2024-12-12 22:30:42.523
2024-12-13 01:00:00,636.785,246.723,2024-12-12 22:30:42.527
2024-12-13 02:00:00,579.516,421.210,2024-12-12 22:30:42.527


In [10]:
mi3_volumes = volumes[volumes['MARKET'] == 'MI3']
mi3_volumes = mi3_volumes.drop(columns=['MARKET'])
mi3_volumes_nord = mi3_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi3_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2024-12-11 18:00:00,100.045,146.434,2024-12-11 10:31:14.247
2024-12-11 19:00:00,111.032,146.895,2024-12-11 10:31:14.247
2024-12-11 20:00:00,150.202,103.458,2024-12-11 10:31:14.247
2024-12-11 21:00:00,111.100,118.125,2024-12-11 10:31:14.247
2024-12-11 22:00:00,101.959,113.528,2024-12-11 10:31:14.247
2024-12-11 23:00:00,121.560,117.964,2024-12-11 10:31:14.247
2024-12-12 12:00:00,203.813,95.622,2024-12-12 10:30:49.400
2024-12-12 13:00:00,300.356,154.474,2024-12-12 10:30:49.403
2024-12-12 14:00:00,333.604,216.584,2024-12-12 10:30:49.403


In [11]:
mi4_volumes = volumes[volumes['MARKET'] == 'MI4']
mi4_volumes = mi4_volumes.drop(columns=['MARKET'])
mi4_volumes_nord = mi4_volumes[['NORD_PURCHASES', 'NORD_SALES', 'LOADTIME']]

mi4_volumes_nord.tail(30)

,NORD_PURCHASES,NORD_SALES,LOADTIME
ORAINI,,,
2021-09-19 10:00:00,35.446,335.960,2021-09-19 04:10:57.840
2021-09-19 11:00:00,27.397,48.069,2021-09-19 04:10:57.840
2021-09-19 12:00:00,13.714,41.316,2021-09-19 04:10:57.843
2021-09-19 13:00:00,20.175,12.943,2021-09-19 04:10:57.843
2021-09-19 14:00:00,27.808,58.166,2021-09-19 04:10:57.843
2021-09-19 15:00:00,17.959,10.590,2021-09-19 04:10:57.843
2021-09-19 16:00:00,42.771,94.523,2021-09-19 04:10:57.843
2021-09-19 17:00:00,110.805,244.656,2021-09-19 04:10:57.843
2021-09-19 18:00:00,120.737,350.911,2021-09-19 04:10:57.843
